## Overview

This notebook shows you how to load data from JDBC databases using Spark SQL.

*For production, you should control the level of parallelism used to read data from the external database, using the parameters described in the documentation.*

### Step 1: Connection Information

This is a **Python** notebook so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` magic command. Python, Scala, SQL, and R are all supported.

First we'll define some variables to let us programmatically create these connections.

In [3]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://database_server"
table = "schema.tablename"
user = ""
password = ""

### Step 2: Reading the data

Now that we specified our file metadata, we can create a DataFrame. You'll notice that we use an *option* to specify that we'd like to infer the schema from the file. We can also explicitly set this to a particular schema if we have one already.

First, let's create a DataFrame in Python, notice how we will programmatically reference the variables we defined above.

In [5]:
remote_table = spark.read.format("jdbc")\
  .option("driver", driver)\
  .option("url", url)\
  .option("dbtable", table)\
  .option("user", user)\
  .option("password", password)\
  .load()

### Step 3: Querying the data

Now that we created our DataFrame. We can query it. For instance, you can select some particular columns to select and display within Databricks.

In [7]:
display(remote_table.select("EXAMPLE_COLUMN"))

### Step 4: (Optional) Create a view or table

If you'd like to be able to use query this data as a table, it is simple to register it as a *view* or a table.

In [9]:
remote_table.createOrReplaceTempView("YOUR_TEMP_VIEW_NAME")

We can query this using Spark SQL. For instance, we can perform a simple aggregation. Notice how we can use `%sql` in order to query the view from SQL.

In [11]:
%sql

SELECT EXAMPLE_GROUP, SUM(EXAMPLE_AGG) FROM YOUR_TEMP_VIEW_NAME GROUP BY EXAMPLE_GROUP

Since this table is registered as a temp view, it will be available only to this notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.

In [13]:
remote_table.write.format("parquet").saveAsTable("MY_PERMANENT_TABLE_NAME")

This table will persist across cluster restarts as well as allow various users across different notebooks to query this data. However, this will not connect back to the original database when doing so.